# **Deploy DeepSeek-Coder-V2 with vLLM on SageMaker Endpoint using LMI container from DJL.**

## Use DJL with the SageMaker Python SDK
- SageMaker Python SDK를 사용하면 Deep Java Library를 이용하여 Amazon SageMaker에서 모델을 호스팅할 수 있습니다. <BR>
- Deep Java Library (DJL) Serving은 DJL이 제공하는 고성능 범용 독립형 모델 서빙 솔루션입니다. DJL Serving은 다양한 프레임워크로 학습된 모델을 로드하는 것을 지원합니다. <BR>
- SageMaker Python SDK를 사용하면 DeepSpeed와 HuggingFace Accelerate와 같은 백엔드를 활용하여 DJL Serving으로 대규모 모델을 호스팅할 수 있습니다. <BR>
- DJL Serving의 지원 버전에 대한 정보는 [AWS 문서](https://docs.aws.amazon.com/deep-learning-containers/latest/devguide/deep-learning-containers-images.html)를 참조하십시오. <BR>
- 최신 지원 버전을 사용하는 것을 권장합니다. 왜냐하면 그곳에 우리의 개발 노력이 집중되어 있기 때문입니다. <BR>
- SageMaker Python SDK 사용에 대한 일반적인 정보는 [SageMaker Python SDK 사용하기](https://sagemaker.readthedocs.io/en/v2.139.0/overview.html#using-the-sagemaker-python-sdk)를 참조하십시오.
    
REF: [BLOG] [Deploy LLM with vLLM on SageMaker in only 13 lines of code](https://mrmaheshrajput.medium.com/deploy-llm-with-vllm-on-sagemaker-in-only-13-lines-of-code-1601f780c0cf)

## 1. Depoly model on SageMaker

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


- [Avalable DLC (Deep Learning Containers)](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)

In [3]:
role = get_execution_role()
region=boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()

region = sagemaker_session.boto_region_name
s3_client = boto3.client("s3", region_name=region)
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtime_client = boto3.client("sagemaker-runtime")
sm_autoscaling_client = boto3.client("application-autoscaling")

### Setup Configuration


In [4]:
model_id = "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct"
#model_id = "deepseek-ai/DeepSeek-Coder-V2-Instruct"

In [5]:
container_uri = sagemaker.image_uris.retrieve(
    framework="djl-lmi", version="0.29.0", region=region
)
if model_id == "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct":
    instance_type = "ml.g5.12xlarge"
elif model_id == "deepseek-ai/DeepSeek-Coder-V2-Instruct":
    instance_type = "ml.p4de.12xlarge"
    
endpoint_name = sagemaker.utils.name_from_base("DeepSeek-Coder-V2-Instruct")

print (f'container_uri: {container_uri}')
print (f'instance_type: {instance_type}')
print (f'endpoint_name: {endpoint_name}')

container_uri: 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.29.0-lmi11.0.0-cu124
instance_type: ml.g5.12xlarge
endpoint_name: DeepSeek-Coder-V2-Instruct-2024-08-26-18-13-10-225


### Creat model with env variables


- Target model: [DeepSeek-Coder-V2-Light-Instruct](https://huggingface.co/deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct)

In [6]:
deploy_env = {
    "HF_MODEL_ID": model_id,
    "OPTION_ROLLING_BATCH": "vllm",
    "OPTION_TENSOR_PARALLEL_DEGREE": "max",
    "OPTION_MAX_ROLLING_BATCH_SIZE": "2",
    "OPTION_DTYPE":"fp16",
    "OPTION_TRUST_REMOTE_CODE": "true",
    "OPTION_MAX_MODEL_LEN": "8192",
}

In [7]:
model = sagemaker.Model(
    image_uri=container_uri, 
    role=role,
    env=deploy_env
)

In [50]:
print(f'model name : {model.name}')

model name : djl-inference-2024-08-26-18-13-10-476


### Deploy model

In [8]:
model.deploy(
    instance_type=instance_type,
    initial_instance_count=1,
    endpoint_name=endpoint_name,
    container_startup_health_check_timeout=900
)

---------------!

In [9]:
print(f'endpoint : {endpoint_name}')

endpoint : DeepSeek-Coder-V2-Instruct-2024-08-26-18-13-10-225


## 2. Invocation (Generate Text using the endpoint)

### Get a predictor for your endpoint

In [10]:
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

### Make a prediction with your endpoint

- **question candidates**
    - write a quick sort algorithm in python.
    - Write a piece of quicksort code in C++.

In [11]:
outputs = predictor.predict(
    {
        "inputs": "write a quick sort algorithm in python.",
        "parameters": {"do_sample": True, "max_new_tokens": 256},
    }
)

print(outputs["generated_text"])

 my code needs some help. 

Here's a simple implementation of the QuickSort algorithm in Python:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quick_sort(left) + middle + quick_sort(right)

# Example usage:
arr = [3, 6, 8, 10, 1, 2, 1]
print(quick_sort(arr))
```

This code defines a function `quick_sort` that recursively sorts an array. It selects a pivot element (in this case, the middle element of the array), then partitions the array into three lists: elements less than the pivot, elements equal to the pivot, and elements greater than the pivot. It then recursively sorts the left and right partitions and concatenates them with the middle list to produce the final sorted array.



In [19]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)

# SageMaker expects resource id to be provided with the following structure
resource_id = f"endpoint/{endpoint_name}/variant/{resp['ProductionVariants'][0]['VariantName']}"

# Scaling configuration
scaling_config_response = sm_autoscaling_client.register_scalable_target(
                                                          ServiceNamespace="sagemaker",
                                                          ResourceId=resource_id,
                                                          ScalableDimension="sagemaker:variant:DesiredInstanceCount", 
                                                          MinCapacity=1,
                                                          MaxCapacity=2
                                                        )

In [20]:
# Create Scaling Policy
policy_name = f"scaling-policy-{endpoint_name}"
scaling_policy_response = sm_autoscaling_client.put_scaling_policy(
                                                PolicyName=policy_name,
                                                ServiceNamespace="sagemaker",
                                                ResourceId=resource_id,
                                                ScalableDimension="sagemaker:variant:DesiredInstanceCount",
                                                PolicyType="TargetTrackingScaling",
                                                TargetTrackingScalingPolicyConfiguration={
                                                    "TargetValue": 5.0, # Target for avg invocations per minutes
                                                    "PredefinedMetricSpecification": {
                                                        "PredefinedMetricType": "SageMakerVariantInvocationsPerInstance",
                                                    },
                                                    "ScaleInCooldown": 600, # Duration in seconds until scale in
                                                    "ScaleOutCooldown": 60 # Duration in seconds between scale out
                                                }
                                            )

In [21]:
import pprint

response = sm_autoscaling_client.describe_scaling_policies(ServiceNamespace="sagemaker")

pp = pprint.PrettyPrinter(indent=4, depth=4)
for i in response["ScalingPolicies"]:
    pp.pprint(i["PolicyName"])
    print("")
    if("TargetTrackingScalingPolicyConfiguration" in i):
        pp.pprint(i["TargetTrackingScalingPolicyConfiguration"])

'scaling-policy-DeepSeek-Coder-V2-Instruct-2024-08-26-18-13-10-225'

{   'PredefinedMetricSpecification': {   'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance'},
    'ScaleInCooldown': 600,
    'ScaleOutCooldown': 60,
    'TargetValue': 5.0}
'scaling-policy-fraud-detect-xgb-endpoint'

{   'PredefinedMetricSpecification': {   'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance'},
    'ScaleInCooldown': 600,
    'ScaleOutCooldown': 60,
    'TargetValue': 5.0}


In [44]:
import time

request_duration = 30
end_time = time.time() + request_duration
print(f"Endpoint will be tested for {request_duration} seconds")
while time.time() < end_time:
    # payload = bytes('{"features": ["write a quick sort algorithm in python."]}', 'utf-8')
    payload = bytes('{"features": ["write a quick sort algorithm in python."]}', 'utf-8')
    response = sm_runtime_client.invoke_endpoint(
                                                 EndpointName=endpoint_name,
                                                 Body=payload,
                                                 ContentType="application/json"
                                                 # Accept="application/json"
                                                )
    # print(response['Body'].read().decode('utf-8'))

Endpoint will be tested for 30 seconds


In [27]:
# Check the instance counts after the endpoint gets more load
response = sm_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_status = response["EndpointStatus"]
request_duration = 250
end_time = time.time() + request_duration
print(f"Waiting for Instance count increase for a max of {request_duration} seconds. Please re run this cell in case the count does not change")
while time.time() < end_time:
    response = sm_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = response["EndpointStatus"]
    instance_count = response["ProductionVariants"][0]["CurrentInstanceCount"]
    print(f"Status: {endpoint_status}")
    print(f"Current Instance count: {instance_count}")
    if (endpoint_status=="InService") and (instance_count>1):
        break
    else:
        time.sleep(15)

Waiting for Instance count increase for a max of 250 seconds. Please re run this cell in case the count does not change
Status: InService
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1


### Streaming output from the endpoint


In [12]:
import io
import json
from pprint import pprint

In [13]:
class LineIterator:
    """
    A helper class for parsing the byte stream input.

    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```

    While usually each PayloadPart event from the event stream will contain a byte array
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```

    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'scan_lines' function. It maintains the position of the last read
    position to ensure that previous bytes are not exposed again.
    """

    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            if line and line[-1] == ord("\n"):
                self.read_pos += len(line)
                return line[:-1]
            try:
                chunk = next(self.byte_iterator)
            except StopIteration:
                if self.read_pos < self.buffer.getbuffer().nbytes:
                    continue
                raise
            if "PayloadPart" not in chunk:
                print("Unknown event type:" + chunk)
                continue
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk["PayloadPart"]["Bytes"])

In [14]:
stop_token = "\n" #Check the stop token for you model

In [15]:
# Create body object and pass 'stream' to True
body = {
    "inputs": "write a quick sort algorithm in python.",
    "parameters": {
        "max_new_tokens": 400,
        # "return_full_text": False  # This does not work with Phi3
    },
    "stream": True,
}

In [16]:
%%time
# Invoke the endpoint
resp = sm_runtime_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name, Body=json.dumps(body), ContentType="application/json"
)

# Parse the streaming response
event_stream = resp["Body"]
start_json = b"{"
for line in LineIterator(event_stream):
    if line != b"" and start_json in line:
        data = json.loads(line[line.find(start_json) :].decode("utf-8"))
        if data["token"]["text"] != stop_token:
            print(data["token"]["text"], end="")

Here's a simple implementation of the Quick Sort algorithm in Python:```pythondef quick_sort(arr):    if len(arr) <= 1:        return arr    else:        pivot = arr[len(arr) // 2]        left = [x for x in arr if x < pivot]        middle = [x for x in arr if x == pivot]        right = [x for x in arr if x > pivot]        return quick_sort(left) + middle + quick_sort(right)# Example usage:arr = [3, 6, 8, 10, 1, 2, 1]print(quick_sort(arr))```This code defines a `quick_sort` function that recursively sorts the input list `arr`. It selects a `pivot` element (in this case, the middle element of the list) and then partitions the list into three parts: elements less than the pivot, elements equal to the pivot, and elements greater than the pivot. The function then recursively sorts the left and right partitions and concatenates them with the middle partition to produce the sorted list.CPU times: user 93.9 ms, sys: 12.2 ms, total: 106 ms
Wall time: 2.88 s


In [17]:
# Create body object and pass 'stream' to True
body = {
    "inputs": "The meaning of life",
    "parameters": {
        "max_new_tokens": 400,
        # "return_full_text": False  # This does not work with Phi3
    },
    "stream": True,
}

In [18]:
# sagemaker_session.delete_endpoint(endpoint_name)